In [1]:
import socket
import os
import threading
import struct

In [2]:
def socket_service():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)# 创建一个新的socket对象
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)# 设置socket选项以便地址和端口可以在程序重启后立即重用
        s.bind(('192.168.11.200', 9002))
        s.listen(10) # ?
    except socket.error as msg:
        print(msg)
        os.sys.exit(1)
    print('等待连接...')

    while True:
        # 循环接收客户端的连接请求。对于每一个连接，创建一个新的线程来处理该连接，确保多个连接可以同时被处理 ?
        conn, addr = s.accept()
        t = threading.Thread(target=deal_data, args=(conn, addr))
        # 使用 threading.Thread 创建新线程来并行处理每个客户端的连接。
        # 这允许服务器同时处理多个客户端请求，提高了服务器的处理能力。
        t.start()

In [ ]:
def deal_data(conn, addr):
    print('客户端地址为： {0}'.format(addr)) 
    try:
        # 读取文件名长度信息
        file_name_length = conn.recv(2) # 接收2字节的数据，这2字节表示文件名的长度
        file_name_length = int.from_bytes(file_name_length, byteorder='big')
        # 指定大端字节序（byteorder='big'），这意味着字节最高位在前

        # 读取文件名
        file_name = conn.recv(file_name_length).decode('utf-8') # 通过文件名长度接收文件名
        # 比如前2字节说明文件名程度为10，那么接下来接收10的长度，这个长度包含了文件名
        print('数据名为 {0}'.format(file_name))

        # 读取文件大小信息
        file_size_bytes = conn.recv(8) # 接收8字节的数据，这8字节表示文件的大小
        file_size = int.from_bytes(file_size_bytes, byteorder='big')
        print('数据大小为 {0}'.format(file_size))

        recvd_size = 0 # 通过文件大小来接收整个文件内容，并写入到服务器的本地文件系统中
        file_path = 'D:/recept/' + file_name
        with open(file_path, 'wb') as fp:
            print('开始接收客户端数据...')
            while recvd_size < file_size:
                data = conn.recv(1024)
                if not data:
                    break
                recvd_size += len(data)
                fp.write(data)
            print('客户端数据接收完成...')

        # 数据接收完毕后，将数据发送回客户端到固定IP和端口
        send_file_to_specific_client(file_path)

    except Exception as e:
        print("处理过程中出现异常: ", e)
    finally:
        conn.close() # 无论处理成功还是发生异常，都确保关闭与客户端的连接
        print('连接已关闭。')

In [3]:
def send_file_to_specific_client(file_path):
    client_ip = '192.168.11.250'
    client_port = 9999
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((client_ip, client_port)) # 连接到指定的客户端IP地址和端口（'192.168.11.250'和端口9999）

        with open(file_path, 'rb') as fp:
            file_name = os.path.basename(file_path)
            file_size = os.stat(file_path).st_size

            file_name_encoded = file_name.encode('utf-8')
            s.send(len(file_name_encoded).to_bytes(2, byteorder='big')) # 发送文件名长度
            s.send(file_name_encoded) # 发送文件名
            s.send(file_size.to_bytes(8, byteorder='big')) # 发送文件大小信息

            print('开始向客户端发送数据...') # 读取文件内容并逐步发送给客户端直至文件末尾
            while True:
                data = fp.read(1024)
                if not data:
                    break
                s.send(data)
            print('数据传输完成：{0}'.format(file_name))
    except Exception as e:
        print("在发送文件过程中发生错误: ", e)
    finally:
        s.close()
        print('文件发送操作完成。')

In [ ]:
if __name__ == '__main__':
    socket_service()